# TCGA Model training
---

Experimenting training models on the preprocessed the TCGA dataset from the Pancancer paper (https://www.ncbi.nlm.nih.gov/pubmed/29625048) into a single, clean dataset.

The Cancer Genome Atlas (TCGA), a landmark cancer genomics program, molecularly characterized over 20,000 primary cancer and matched normal samples spanning 33 cancer types. This joint effort between the National Cancer Institute and the National Human Genome Research Institute began in 2006, bringing together researchers from diverse disciplines and multiple institutions.

## Importing the necessary packages

In [1]:
import os                                  # os handles directory/workspace changes
import torch                               # PyTorch to create and apply deep learning models
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, f1_score, log_loss, roc_auc_score
import sys

In [2]:
# Path to the dataset files
data_path = 'cleaned/'

In [3]:
import modin.pandas as pd                  # Optimized distributed version of Pandas

Allow pandas to show more columns:

In [4]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

Set the random seed for reproducibility:

In [5]:
import numpy as np
import pandas
np.random.seed(42)

## Loading the data

In [6]:
tcga_df = pandas.read_csv(f'{data_path}normalized/tcga.csv')
tcga_df.head()

,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,ABCA13|154664,ABCA17P|650655,ABCA2|20,ABCA3|21,ABCA4|24,ABCA5|23461,ABCA6|23460,ABCA7|10347,ABCA8|10351,ABCA9|10350,ABCB10|23456,ABCB11|8647,ABCB1|5243,ABCB4|5244,ABCB5|340273,ABCB6|10058,ABCB7|22,ABCB8|11194,ABCB9|23457,ABCC10|89845,ABCC11|85320,ABCC12|94160,ABCC13|150000,ABCC1|4363,ABCC2|1244,ABCC3|8714,ABCC4|10257,ABCC5|10057,ABCC6|368,ABCC6P1|653190,ABCC6P2|730013,ABCC8|6833,ABCC9|10060,ABCD1|215,ABCD2|225,ABCD3|5825,ABCD4|5826,ABCE1|6059,ABCF1|23,ABCF2|10061,ABCF3|55324,ABCG1|9619,ABCG2|9429,ABCG4|64137,ABCG5|64240,ABCG8|64241,ABHD10|55347,ABHD11|83451,ABHD12|26090,ABHD12B|145447,ABHD13|84945,ABHD14A|25864,ABHD14B|84836,ABHD15|116236,ABHD1|84696,ABHD2|11057,ABHD3|171586,ABHD4|63874,ABHD5|51099,ABHD6|57406,ABHD8|79575,ABI1|10006,ABI2|10152,ABI3|51225,ABI3BP|25890,ABL1|25,ABL2|27,ABLIM1|3983,ABLIM2|84448,ABLIM3|22885,ABO|28,ABP1|26,ABR|29,ABRA|137735,ABT1|29777,ABTB1|80325,ABTB2|25841,ACAA1|30,ACAA2|10449,ACACA|31,ACACB|32,ACAD10|80724,ACAD11|84129,ACAD8|27034,ACAD9|28976,ACADL|33,ACADM|34,ACADS|35,ACADSB|36,ACADVL|37,ACAN|176,ACAP1|9744,ACAP2|23527,ACAP3|116983,ACAT1|38,ACAT2|39,ACBD3|64746,ACBD4|79777,ACBD5|91452,ACBD6|84320,ACBD7|414149,ACCN1|40,ACCN2|41,ACCN3|9311,ACCN4|55515,ACCN5|51802,ACCS|84680,ACCSL|390110,ACD|65057,ACE|1636,ACE2|59272,ACER1|125981,ACER2|340485,ACER3|55331,ACHE|43,ACIN1|22985,ACLY|47,ACMSD|130013,ACN9|57001,ACO1|48,ACO2|50,ACOT11|26027,ACOT12|134526,ACOT13|55856,ACOT1|641371,ACOT2|10965,ACOT4|122970,ACOT6|641372,ACOT7|11332,ACOT8|10005,ACOT9|23597,ACOX1|51,ACOX2|8309,ACOX3|8310,ACOXL|55289,ACP1|52,ACP2|53,ACP5|54,ACP6|51205,ACPL2|92370,ACPP|55,ACPT|93650,ACR|49,ACRBP|84519,ACRC|93953,ACRV1|56,ACSBG1|23205,ACSBG2|81616,ACSF2|80221,ACSF3|197322,ACSL1|2180,ACSL3|2181,ACSL4|2182,ACSL5|51703,ACSL6|23305,ACSM1|116285,ACSM2A|123876,ACSM2B|348158,ACSM3|6296,ACSM4|341392,ACSM5|54988,ACSS1|84532,ACSS2|55902,ACSS3|79611,ACTA1|58,ACTA2|59,ACTB|60,ACTBL2|345651,ACTC1|70,ACTG1|71,ACTG2|72,ACTL6A|86,ACTL6B|51412,ACTL7A|10881,ACTL7B|10880,ACTL8|81569,ACTL9|284382,ACTN1|87,ACTN2|88,ACTN3|89,ACTN4|81,ACTR10|55860,ACTR1A|10121,ACTR1B|10120,ACTR2|10097,ACTR3|10096,ACTR3B|57180,ACTR3C|653857,ACTR5|79913,ACTR6|64431,ACTR8|93973,ACTRT1|139741,ACTRT2|140625,ACVR1|90,ACVR1B|91,ACVR1C|130399,ACVR2A|92,ACVR2B|93,ACVRL1|94,ACY1|95,ACY3|91703,ACYP1|97,ACYP2|98,ADA|100,ADAD1|132612,ADAD2|161931,ADAL|161823,ADAM10|102,ADAM11|4185,ADAM12|8038,ADAM15|8751,ADAM17|6868,ADAM18|8749,ADAM19|8728,ADAM20|8748,ADAM21|8747,ADAM21P1|145241,ADAM2|2515,ADAM22|53616,ADAM23|8745,ADAM28|10863,ADAM29|11086,ADAM30|11085,ADAM32|203102,ADAM33|80332,ADAM3A|1587,ADAM5P|255926,ADAM6|8755,ADAM7|8756,ADAM8|101,ADAM9|8754,ADAMDEC1|27299,ADAMTS10|81794,ADAMTS12|81792,ADAMTS13|11093,ADAMTS14|140766,ADAMTS15|170689,ADAMTS16|170690,ADAMTS17|170691,ADAMTS18|170692,ADAMTS19|171019,ADAMTS1|9510,ADAMTS20|80070,ADAMTS2|9509,ADAMTS3|9508,ADAMTS4|9507,ADAMTS5|11096,ADAMTS6|11174,ADAMTS7|11173,ADAMTS8|11095,ADAMTS9|56999,ADAMTSL1|92949,ADAMTSL2|9719,ADAMTSL3|57188,ADAMTSL4|54507,ADAMTSL5|339366,ADAP1|11033,ADAP2|55803,ADAR|103,ADARB1|104,ADARB2|105,ADAT1|23536,ADAT2|134637,ADAT3|113179,ADC|113451,ADCK1|57143,ADCK2|90956,ADCK4|79934,ADCK5|203054,ADCY10|55811,ADCY1|107,ADCY2|108,ADCY3|109,ADCY4|196883,ADCY5|111,ADCY6|112,ADCY7|113,ADCY8|114,ADCY9|115,ADCYAP1|116,ADCYAP1R1|117,ADD1|118,ADD2|119,ADD3|120,ADH1A|124,ADH1B|125,ADH1C|1

In [7]:
tcga_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4628 entries, 0 to 4627
Columns: 20829 entries, ?|100130426 to treatment_outcome_first_course
dtypes: float64(20819), int64(3), object(7)
memory usage: 735.4+ MB


In [8]:
tcga_df.participant_id.value_counts()

participant_id
TCGA-02-0047    1
TCGA-EW-A1IW    1
TCGA-F4-6570    1
TCGA-F4-6569    1
TCGA-F4-6463    1
               ..
TCGA-BP-5000    1
TCGA-BP-4999    1
TCGA-BP-4998    1
TCGA-BP-4994    1
TCGA-ZX-AA5X    1
Name: count, Length: 4628, dtype: int64

In [9]:
tcga_df.dtypes

?|100130426                       float64
?|100133144                       float64
?|100134869                       float64
?|10357                           float64
?|10431                           float64
                                   ...   
clinical_stage                     object
histological_grade                 object
menopause_status                   object
cause_of_death                     object
treatment_outcome_first_course     object
Length: 20829, dtype: object

Remove the original string ID column and use the numeric one instead:

In [10]:
tcga_df = tcga_df.drop(columns=['participant_id'], axis=1)
tcga_df = tcga_df.rename(columns={'Unnamed: 0': 'sample_id'})
tcga_df.head()

,?|100130426,?|100133144,?|100134869,?|10357,?|10431,?|136542,?|155060,?|26823,?|280660,?|317712,?|340602,?|388795,?|390284,?|391343,?|391714,?|404770,?|441362,?|442388,?|553137,?|57714,?|645851,?|652919,?|653553,?|728045,?|728603,?|728788,?|729884,?|8225,?|90288,A1BG|1,A1CF|29974,A2BP1|54715,A2LD1|87769,A2M|2,A2ML1|144568,A4GALT|53947,A4GNT|51146,AAA1|404744,AAAS|8086,AACS|65985,AACSL|729522,AADAC|13,AADACL2|344752,AADACL3|126767,AADACL4|343066,AADAT|51166,AAGAB|79719,AAK1|22848,AAMP|14,AANAT|15,AARS|16,AARS2|57505,AARSD1|80755,AASDH|132949,AASDHPPT|60496,AASS|10157,AATF|26574,AATK|9625,ABAT|18,ABCA10|10349,ABCA1|19,ABCA11P|79963,ABCA12|26154,ABCA13|154664,ABCA17P|650655,ABCA2|20,ABCA3|21,ABCA4|24,ABCA5|23461,ABCA6|23460,ABCA7|10347,ABCA8|10351,ABCA9|10350,ABCB10|23456,ABCB11|8647,ABCB1|5243,ABCB4|5244,ABCB5|340273,ABCB6|10058,ABCB7|22,ABCB8|11194,ABCB9|23457,ABCC10|89845,ABCC11|85320,ABCC12|94160,ABCC13|150000,ABCC1|4363,ABCC2|1244,ABCC3|8714,ABCC4|10257,ABCC5|10057,ABCC6|368,ABCC6P1|653190,ABCC6P2|730013,ABCC8|6833,ABCC9|10060,ABCD1|215,ABCD2|225,ABCD3|5825,ABCD4|5826,ABCE1|6059,ABCF1|23,ABCF2|10061,ABCF3|55324,ABCG1|9619,ABCG2|9429,ABCG4|64137,ABCG5|64240,ABCG8|64241,ABHD10|55347,ABHD11|83451,ABHD12|26090,ABHD12B|145447,ABHD13|84945,ABHD14A|25864,ABHD14B|84836,ABHD15|116236,ABHD1|84696,ABHD2|11057,ABHD3|171586,ABHD4|63874,ABHD5|51099,ABHD6|57406,ABHD8|79575,ABI1|10006,ABI2|10152,ABI3|51225,ABI3BP|25890,ABL1|25,ABL2|27,ABLIM1|3983,ABLIM2|84448,ABLIM3|22885,ABO|28,ABP1|26,ABR|29,ABRA|137735,ABT1|29777,ABTB1|80325,ABTB2|25841,ACAA1|30,ACAA2|10449,ACACA|31,ACACB|32,ACAD10|80724,ACAD11|84129,ACAD8|27034,ACAD9|28976,ACADL|33,ACADM|34,ACADS|35,ACADSB|36,ACADVL|37,ACAN|176,ACAP1|9744,ACAP2|23527,ACAP3|116983,ACAT1|38,ACAT2|39,ACBD3|64746,ACBD4|79777,ACBD5|91452,ACBD6|84320,ACBD7|414149,ACCN1|40,ACCN2|41,ACCN3|9311,ACCN4|55515,ACCN5|51802,ACCS|84680,ACCSL|390110,ACD|65057,ACE|1636,ACE2|59272,ACER1|125981,ACER2|340485,ACER3|55331,ACHE|43,ACIN1|22985,ACLY|47,ACMSD|130013,ACN9|57001,ACO1|48,ACO2|50,ACOT11|26027,ACOT12|134526,ACOT13|55856,ACOT1|641371,ACOT2|10965,ACOT4|122970,ACOT6|641372,ACOT7|11332,ACOT8|10005,ACOT9|23597,ACOX1|51,ACOX2|8309,ACOX3|8310,ACOXL|55289,ACP1|52,ACP2|53,ACP5|54,ACP6|51205,ACPL2|92370,ACPP|55,ACPT|93650,ACR|49,ACRBP|84519,ACRC|93953,ACRV1|56,ACSBG1|23205,ACSBG2|81616,ACSF2|80221,ACSF3|197322,ACSL1|2180,ACSL3|2181,ACSL4|2182,ACSL5|51703,ACSL6|23305,ACSM1|116285,ACSM2A|123876,ACSM2B|348158,ACSM3|6296,ACSM4|341392,ACSM5|54988,ACSS1|84532,ACSS2|55902,ACSS3|79611,ACTA1|58,ACTA2|59,ACTB|60,ACTBL2|345651,ACTC1|70,ACTG1|71,ACTG2|72,ACTL6A|86,ACTL6B|51412,ACTL7A|10881,ACTL7B|10880,ACTL8|81569,ACTL9|284382,ACTN1|87,ACTN2|88,ACTN3|89,ACTN4|81,ACTR10|55860,ACTR1A|10121,ACTR1B|10120,ACTR2|10097,ACTR3|10096,ACTR3B|57180,ACTR3C|653857,ACTR5|79913,ACTR6|64431,ACTR8|93973,ACTRT1|139741,ACTRT2|140625,ACVR1|90,ACVR1B|91,ACVR1C|130399,ACVR2A|92,ACVR2B|93,ACVRL1|94,ACY1|95,ACY3|91703,ACYP1|97,ACYP2|98,ADA|100,ADAD1|132612,ADAD2|161931,ADAL|161823,ADAM10|102,ADAM11|4185,ADAM12|8038,ADAM15|8751,ADAM17|6868,ADAM18|8749,ADAM19|8728,ADAM20|8748,ADAM21|8747,ADAM21P1|145241,ADAM2|2515,ADAM22|53616,ADAM23|8745,ADAM28|10863,ADAM29|11086,ADAM30|11085,ADAM32|203102,ADAM33|80332,ADAM3A|1587,ADAM5P|255926,ADAM6|8755,ADAM7|8756,ADAM8|101,ADAM9|8754,ADAMDEC1|27299,ADAMTS10|81794,ADAMTS12|81792,ADAMTS13|11093,ADAMTS14|140766,ADAMTS15|170689,ADAMTS16|170690,ADAMTS17|170691,ADAMTS18|170692,ADAMTS19|171019,ADAMTS1|9510,ADAMTS20|80070,ADAMTS2|9509,ADAMTS3|9508,ADAMTS4|9507,ADAMTS5|11096,ADAMTS6|11174,ADAMTS7|11173,ADAMTS8|11095,ADAMTS9|56999,ADAMTSL1|92949,ADAMTSL2|9719,ADAMTSL3|57188,ADAMTSL4|54507,ADAMTSL5|339366,ADAP1|11033,ADAP2|55803,ADAR|103,ADARB1|104,ADARB2|105,ADAT1|23536,ADAT2|134637,ADAT3|113179,ADC|113451,ADCK1|57143,ADCK2|90956,ADCK4|79934,ADCK5|203054,ADCY10|55811,ADCY1|107,ADCY2|108,ADCY3|109,ADCY4|196883,ADCY5|111,ADCY6|112,ADCY7|113,ADCY8|114,ADCY9|115,ADCYAP1|116,ADCYAP1R1|117,ADD1|118,ADD2|119,ADD3|120,ADH1A|124,ADH1B|125,ADH1C|1

Convert the label to a numeric format:

In [11]:
tcga_df.tumor_type_label.value_counts()

tumor_type_label
BRCA    1021
LGG      500
HNSC     485
BLCA     397
COAD     389
KIRC     328
LIHC     310
CESC     291
KIRP     270
ESCA     161
GBM      147
LAML     109
ACC       76
KICH      61
DLBC      47
CHOL      36
Name: count, dtype: int64

In [12]:
# Encode the categorical labels into numeric values
tcga_df['tumor_type_label'], label_array = pandas.factorize(tcga_df['tumor_type_label'])

# Create a dictionary mapping integer labels to original string labels
label_dict = {i: label for i, label in enumerate(label_array)}

# Show value counts of the encoded labels
tcga_df['tumor_type_label'].value_counts()


tumor_type_label
6     1021
12     500
10     485
1      397
7      389
9      328
4      310
5      291
3      270
2      161
0      147
11     109
15      76
14      61
13      47
8       36
Name: count, dtype: int64

In [13]:
label_dict

{0: 'GBM',
 1: 'BLCA',
 2: 'ESCA',
 3: 'KIRP',
 4: 'LIHC',
 5: 'CESC',
 6: 'BRCA',
 7: 'COAD',
 8: 'CHOL',
 9: 'KIRC',
 10: 'HNSC',
 11: 'LAML',
 12: 'LGG',
 13: 'DLBC',
 14: 'KICH',
 15: 'ACC'}

In [14]:
tcga_df.dtypes

?|100130426                       float64
?|100133144                       float64
?|100134869                       float64
?|10357                           float64
?|10431                           float64
                                   ...   
clinical_stage                     object
histological_grade                 object
menopause_status                   object
cause_of_death                     object
treatment_outcome_first_course     object
Length: 20828, dtype: object

Convert to a PyTorch tensor:

In [15]:
tcga_df.dtypes[tcga_df.dtypes == 'object']
for col in tcga_df.select_dtypes(include='object').columns:
    tcga_df[col] = pandas.factorize(tcga_df[col])[0]

In [16]:
tcga_df = tcga_df.astype(np.float32)  # or np.float64 if preferred


In [17]:
tcga_tsr = torch.from_numpy(tcga_df.to_numpy())
tcga_tsr

tensor([[-0.1186, -0.6704, -0.4207,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1186, -0.9223,  0.3450,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1186, -0.6808,  0.1944,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000, -0.1473, -0.6443,  ...,  0.0000,  0.0000,  3.0000],
        [-0.1186, -0.8042, -0.7817,  ...,  0.0000,  0.0000,  0.0000],
        [-0.1186,  0.0483,  1.5638,  ...,  2.0000,  1.0000,  5.0000]])

Create a dataset:

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.decomposition import TruncatedSVD

# Step 1: Filter out low-frequency classes (fewer than 10 samples)
min_class_samples = 10
class_counts = tcga_df['tumor_type_label'].value_counts()
valid_classes = class_counts[class_counts >= min_class_samples].index.tolist()
filtered_df = tcga_df[tcga_df['tumor_type_label'].isin(valid_classes)].copy()

# Step 2: Re-encode labels to 0..K format
filtered_df['tumor_type_label'], label_array = pandas.factorize(filtered_df['tumor_type_label'])
label_dict = {i: label for i, label in enumerate(label_array)}

# Step 3: Convert object columns to numeric
for col in filtered_df.select_dtypes(include='object').columns:
    filtered_df[col] = pandas.factorize(filtered_df[col])[0]

# Step 4: Convert to float32
filtered_df = filtered_df.astype(np.float32)

# Step 5: Extract features and labels
X_all = filtered_df.drop(columns=['tumor_type_label']).to_numpy()
y_all = filtered_df['tumor_type_label'].to_numpy().astype(int)

# Step 6: Stratified split into train/cal/test (70/10/20)
X_temp, X_test, y_temp, y_test = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42, stratify=y_all)

X_train, X_cal, y_temp_train, y_temp_cal = train_test_split(
    X_temp, y_temp, test_size=0.3, random_state=42, stratify=y_temp)  # 0.125 of 0.8 = 10%

# Step 7: Re-encode training labels and align others to the same mapping
y_train, label_array = pandas.factorize(y_temp_train)
label_dict = {i: label for i, label in enumerate(label_array)}
label_map = {v: i for i, v in enumerate(label_array)}

y_cal = np.array([label_map[y] for y in y_temp_cal])
y_test = np.array([label_map[y] for y in y_test if y in label_map])
X_test = X_test[:len(y_test)]
X_cal = X_cal[:len(y_cal)]

# Step 8: Apply Truncated SVD to reduce dimensionality
svd = TruncatedSVD(n_components=10, random_state=42)
X_train_reduced = svd.fit_transform(X_train)
X_cal_reduced = svd.transform(X_cal)
X_test_reduced = svd.transform(X_test)


In [66]:
def get_batches(X, y, batch_size):
    for i in range(0, len(X), batch_size):
        X_batch = torch.tensor(X[i:i+batch_size]).float()
        y_batch = torch.tensor(y[i:i+batch_size]).long()
        yield X_batch, y_batch


## Training models

In [67]:
from sklearn.decomposition import TruncatedSVD
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis


In [68]:
# Initialize LDA
lda_model = LinearDiscriminantAnalysis()

# Train LDA
lda_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = lda_model.predict(X_test)
y_proba = lda_model.predict_proba(X_test)

lda_acc = accuracy_score(y_test, y_pred)

print(f"LDA → Accuracy: {lda_acc:.4f}")

lda_model.fit(X_train_reduced, y_train)
y_pred = lda_model.predict(X_test_reduced)
lda_acc = accuracy_score(y_test, y_pred)

print(f"LDA with Dimensionality Reduction → Accuracy: {lda_acc:.4f}")

LDA → Accuracy: 0.9633
LDA with Dimensionality Reduction → Accuracy: 0.8380


In [69]:
# Initialize QDA
qda_model = QuadraticDiscriminantAnalysis()

# Train QDA
qda_model.fit(X_train, y_train)

# Predict and evaluate
y_pred = qda_model.predict(X_test)
y_proba = qda_model.predict_proba(X_test)

qda_acc = accuracy_score(y_test, y_pred)

print(f"QDA → Accuracy: {qda_acc:.4f}")

qda_model.fit(X_train_reduced, y_train)
y_pred = qda_model.predict(X_test_reduced)
qda_acc = accuracy_score(y_test, y_pred)

print(f"QDA with Dimensionality Reduction → Accuracy: {qda_acc:.4f}")

QDA → Accuracy: 0.1037
QDA with Dimensionality Reduction → Accuracy: 0.9006


# CONFORMAL PREDICTION

In [70]:
from mapie.classification import SplitConformalClassifier
from mapie.metrics.classification import classification_coverage_score, classification_mean_width_score


In [79]:
alpha = 0.01

# Create SplitConformalClassifier wrapper
mapie_lda = SplitConformalClassifier(
    estimator=lda_model,
    conformity_score="aps",
    confidence_level=1 - alpha, 
    prefit=True,
    random_state=42
)

# Calibrate using reduced calibration set
mapie_lda.conformalize(X_cal_reduced, y_cal)

# Predict conformal prediction sets on test set
_, y_pset_lda = mapie_lda.predict_set(X_test_reduced)

# Evaluate coverage and prediction set width
coverage_lda = classification_coverage_score(y_test, y_pset_lda)
width_lda = classification_mean_width_score(y_pset_lda)

print(f"LDA (APS, alpha={alpha:.2f}) → Coverage: {coverage_lda.item():.4f}, Avg Set Size: {width_lda.item():.2f}")



LDA (APS, alpha=0.01) → Coverage: 0.9903, Avg Set Size: 3.33


In [80]:
# Create SplitConformalClassifier wrapper
mapie_qda = SplitConformalClassifier(
    estimator=qda_model,
    conformity_score="aps",
    confidence_level=1 - alpha, 
    prefit=True,
    random_state=42
)

# Calibrate using reduced calibration set
mapie_qda.conformalize(X_cal_reduced, y_cal)

# Predict conformal prediction sets on test set
_, y_pset_qda = mapie_qda.predict_set(X_test_reduced)

# Evaluate coverage and prediction set width
coverage_qda = classification_coverage_score(y_test, y_pset_qda)
width_qda = classification_mean_width_score(y_pset_qda)

print(f"QDA (APS, alpha={alpha:.2f}) → Coverage: {coverage_qda.item():.4f}, Avg Set Size: {width_qda.item():.2f}")



QDA (APS, alpha=0.01) → Coverage: 0.9935, Avg Set Size: 2.60


In [81]:
print("Number of unique labels in y_test:", len(np.unique(y_test)))
print("Labels in y_test:", np.unique(y_test))


Number of unique labels in y_test: 16
Labels in y_test: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]


In [82]:
def brier_score(p, y_true):
    y_onehot = np.zeros_like(p)
    y_onehot[np.arange(len(y_true)), y_true] = 1
    return np.mean(np.sum((y_onehot - p)**2, axis=1))

def spherical_score(p, y_true):
    norms = np.linalg.norm(p, axis=1)
    return np.mean([p[i, y_true[i]] / norms[i] for i in range(len(y_true))])

def log_score(p, y_true):
    return -np.mean(np.log(p[np.arange(len(y_true)), y_true] + 1e-15))

### For LDA
probs_lda = lda_model.predict_proba(X_test_reduced)
logloss_lda = log_score(probs_lda, y_test)
brier_lda = brier_score(probs_lda, y_test)
spherical_lda = spherical_score(probs_lda, y_test)

print("\nLDA Proper Scoring Rules:")
print(f"Log Loss:        {logloss_lda:.4f}")
print(f"Brier Score:     {brier_lda:.4f}")
print(f"Spherical Score: {spherical_lda:.4f}")

### For QDA
probs_qda = qda_model.predict_proba(X_test_reduced)
logloss_qda = log_score(probs_qda, y_test)
brier_qda = brier_score(probs_qda, y_test)
spherical_qda = spherical_score(probs_qda, y_test)

print("\nQDA Proper Scoring Rules:")
print(f"Log Loss:        {logloss_qda:.4f}")
print(f"Brier Score:     {brier_qda:.4f}")
print(f"Spherical Score: {spherical_qda:.4f}")



LDA Proper Scoring Rules:
Log Loss:        0.8382
Brier Score:     0.2513
Spherical Score: 0.8560

QDA Proper Scoring Rules:
Log Loss:        0.5609
Brier Score:     0.1605
Spherical Score: 0.9103
